# Задание 1

In [1]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()
sql = """
    CREATE TABLE DataTable1 (
        title TEXT,
        author TEXT,
        genre TEXT,
        pages INTEGER, 
        published_year INTEGER,
        isbn TEXT,
        rating FLOAT,
        views INTEGER
        )"""

cur.execute(sql)

In [2]:
with open('./1/task_1_var_02_item.text', 'r', encoding='utf-8') as f:
    lines = f.readlines()

for item in [[lines[i*9+j].strip().split('::')[-1] for j in range(8)] for i in range(len(lines)//9)]:
    cur.execute("""INSERT INTO DataTable1 VALUES (
        :title,
        :author,
        :genre,
        :pages, 
        :published_year,
        :isbn,
        :rating,
        :views)""", item)
conn.commit()

In [3]:
import json

def get_from_head(conn, sort_col, n):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT title,
        author,
        genre,
        pages, 
        published_year,
        isbn,
        rating,
        views 
        FROM DataTable1 ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head(conn, 'rating', 12)

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("1_top.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [4]:
def get_stats(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        SUM({col}), MIN({col}), MAX({col}), AVG({col})  
        FROM DataTable1""")
    return stats.fetchone()

stats = get_stats(conn, 'rating')
keys = ['sum', 'min', 'max', 'avg']

with open("1_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump({keys[i]: str(e) for i, e in enumerate(stats)}, outfile, ensure_ascii=True)

In [5]:
def get_freq(conn, col):
    cur = conn.cursor()
    freq = cur.execute(f"""SELECT 
        {col}, COUNT(*)
        FROM DataTable1 GROUP BY {col}""")
    return freq

freq = get_freq(conn, 'genre')
freq = {e[0]: str(e[1]) for e in freq}

with open("1_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

In [6]:
def get_from_head_with_condition(conn, sort_col, n, condition):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT title,
        author,
        genre,
        pages, 
        published_year,
        isbn,
        rating,
        views 
        FROM DataTable1 WHERE {condition} ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head_with_condition(conn, 'rating', 12, 'published_year > 1900')

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("1_top_with_condition.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

# Задание 2

In [7]:
sql = """
    CREATE TABLE DataTable2 (
        title TEXT,
        price INTEGER,
        place TEXT,
        data DATETIME
        )"""

cur.execute(sql)

In [8]:
import msgpack
    
with open("./2/task_2_var_02_subitem.msgpack", "rb") as data_file:
    byte_data = data_file.read()
    data = msgpack.unpackb(byte_data)
data[0]

{'title': 'Дюна', 'price': 4368, 'place': 'offline', 'date': '3.5.2014'}

In [9]:
for item in data:
    cur.execute("""INSERT INTO DataTable2 VALUES (
        :title,
        :price,
        :place,
        :data)""", list(item.values()))
conn.commit()

In [10]:
def get_from_head_both(conn, n):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT DataTable1.title,
        DataTable1.author,
        DataTable1.genre,
        DataTable1.pages, 
        DataTable1.published_year,
        DataTable1.isbn,
        DataTable1.rating,
        DataTable1.views,
        DataTable2.price,
        DataTable2.place,
        DataTable2.data        
        FROM DataTable1 JOIN DataTable2 ON DataTable1.title = DataTable2.title""")
    return samples


cur = get_from_head_both(conn, 12)

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views', 'price', 'place', 'data']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("2_top_from_both.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [11]:
def get_from_head_both_with_condition(conn, n, condition):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT DataTable1.title,
        DataTable1.author,
        DataTable1.genre,
        DataTable1.pages, 
        DataTable1.published_year,
        DataTable1.isbn,
        DataTable1.rating,
        DataTable1.views,
        DataTable2.price,
        DataTable2.place,
        DataTable2.data        
        FROM DataTable1 JOIN DataTable2 ON DataTable1.title = DataTable2.title
        WHERE {condition}""")
    return samples


cur = get_from_head_both_with_condition(conn, 12, 'DataTable2.price > 1000')

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views', 'price', 'place', 'data']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("2_top_from_both_with_condition.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [12]:
def get_from_head_both_ordered(conn, n, sort_col):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT DataTable1.title,
        DataTable1.author,
        DataTable1.genre,
        DataTable1.pages, 
        DataTable1.published_year
        FROM DataTable1 JOIN DataTable2 ON DataTable1.title = DataTable2.title
        ORDER BY {sort_col}""")
    return samples


cur = get_from_head_both_ordered(conn, 12, 'DataTable2.price')

keys = ['title', 'author', 'genre', 'pages', 'published_year']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("2_top_from_both_ordered.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

# Задание 3

In [13]:
with open("3/task_3_var_02_part_1.json", encoding='utf-8') as f:
    part1 = json.load(f)
    
import msgpack
    
with open("3/task_3_var_02_part_2.msgpack", "rb") as data_file:
    byte_data = data_file.read()
    part2 = msgpack.unpackb(byte_data)
len(part1), len(part2)

(1198, 802)

In [14]:
common_keys = ['artist', 'song', 'duration_ms', 'year', 'tempo', 'genre']

result = [{key: e[key] for key in common_keys} for e in part1] + [{key: e[key] for key in common_keys} for e in part2]
len(result)

2000

In [15]:
sql = """
    CREATE TABLE DataTable3 (
        artist TEXT, 
        song TEXT, 
        duration_ms INTEGER,
        year INTEGER,
        tempo FLOAT, 
        genre TEXT
        )"""

cur.execute(sql)

In [16]:
for item in result:
    cur.execute("""INSERT INTO DataTable3 VALUES (
        :artist,
        :song,
        :duration_ms,
        :year, 
        :tempo,
        :genre)""", list(item.values()))
conn.commit()

In [17]:
def get_from_head(conn, sort_col, n):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT artist,
        song,
        duration_ms,
        year, 
        tempo,
        genre
        FROM DataTable3 ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head(conn, 'year', 12)

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("3_top.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [18]:
def get_stats(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        SUM({col}), MIN({col}), MAX({col}), AVG({col})  
        FROM DataTable3""")
    return stats.fetchone()

stats = get_stats(conn, 'duration_ms')
keys = ['sum', 'min', 'max', 'avg']

with open("3_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump({keys[i]: str(e) for i, e in enumerate(stats)}, outfile, ensure_ascii=True)

In [19]:
def get_freq(conn, col):
    cur = conn.cursor()
    freq = cur.execute(f"""SELECT 
        {col}, COUNT(*)
        FROM DataTable3 GROUP BY {col}""")
    return freq

freq = get_freq(conn, 'artist')
freq = {e[0]: str(e[1]) for e in freq}

with open("3_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

In [20]:
def get_from_head_with_condition(conn, sort_col, n, condition):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT artist,
        song,
        duration_ms,
        year, 
        tempo,
        genre
        FROM DataTable3 WHERE {condition} ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head_with_condition(conn, 'year', 17, 'genre == "rock, pop"')

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("3_top_with_condition.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

# Задание 4

In [21]:
import pickle

with open('./4/task_4_var_02_product_data.pkl', "rb") as f:
    product_data = pickle.load(f)
with open('./4/task_4_var_02_update_data.json', encoding='utf-8') as f:
    update_data = json.load(f)

In [22]:
import numpy as np

def fill_dict(d, keys):
    for key in keys:
        if key not in d:
            d[key] = None
    return d


keys = np.unique([key for e in product_data for key in e.keys()])
product_data = [fill_dict(e, keys) for e in product_data]

In [23]:
sql = """
    CREATE TABLE DataTable4 (
        name TEXT, 
        price FLOAT, 
        quantity INTEGER,
        category TEXT,
        fromCity TEXT, 
        isAvailable BOOLEAN,
        views INTEGER,
        updates INTEGER 
        )"""

cur.execute(sql)

In [24]:
for item in product_data:
    cur.execute("""INSERT INTO DataTable4 VALUES (
        :name, 
        :price, 
        :quantity,
        :category,
        :fromCity, 
        :isAvailable,
        :views,
        :updates)""", list(item.values()) + [0])
conn.commit()

In [25]:
np.unique([e['method'] for e in update_data])

array(['available', 'price_abs', 'price_percent', 'quantity_add',
       'quantity_sub', 'remove'], dtype='<U13')

In [26]:
def update_available(conn, col, param):
    cur = conn.cursor()
    res = cur.execute(f"UPDATE DataTable4 SET isAvailable = {param} WHERE (name = \"{col}\")")
    cur.execute(f"UPDATE DataTable4 SET updates = updates + 1 WHERE name = \"{col}\"")
    conn.commit()
    

def update_price_abs(conn, col, param):
    cur = conn.cursor()
    if param[0] != '-':
        param = '+' + param
    res = cur.execute(f"UPDATE DataTable4 SET price = (price {param}) WHERE (name = \"{col}\") AND ((price {param}) > 0)")
    if res.rowcount > 0:
        cur.execute(f"UPDATE DataTable4 SET updates = updates + 1 WHERE name = \"{col}\"")
        conn.commit()

    
def update_price_percent(conn, col, param):
    cur = conn.cursor()
    res = cur.execute(f"UPDATE DataTable4 SET price = ROUND((price * (1 + {param})), 2) WHERE (name = \"{col}\") AND ((1 + {param}) > 0)")
    if res.rowcount > 0:
        cur.execute(f"UPDATE DataTable4 SET updates = updates + 1 WHERE name = \"{col}\"")
        conn.commit()

    
def update_quantity(conn, col, param):
    cur = conn.cursor()
    if param[0] != '-':
        param = '+' + param
    res = cur.execute(f"UPDATE DataTable4 SET quantity = (quantity {param}) WHERE (name = \"{col}\") AND ((quantity {param}) > 0)")
    if res.rowcount > 0:
        cur.execute(f"UPDATE DataTable4 SET updates = updates + 1 WHERE name = \"{col}\"")
        conn.commit()
    
    
def delete_by_col(conn, col):
    cur = conn.cursor()
    cur.execute(f"DELETE FROM DataTable4 WHERE name = \"{col}\"")
    conn.commit()

In [27]:
for update in update_data:
    name, method, param = update.values()
    param = str(param)
    if method == 'available':
        update_available(conn, name, param)
    elif method == 'price_abs':
        update_price_abs(conn, name, param)
    elif method == 'price_percent':
        update_price_percent(conn, name, param)
    elif method == 'quantity_add':
        update_quantity(conn, name, param)
    elif method == 'quantity_sub':
        update_quantity(conn, name, param)
    elif method == 'remove':
        delete_by_col(conn, name)

In [28]:
def get_from_head(conn, sort_col, n):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT 
        name, 
        price, 
        quantity,
        category,
        fromCity, 
        isAvailable,
        views,
        updates
        FROM DataTable4 ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head(conn, 'updates', 10)

keys = ['name', 'price', 'quantity', 'category', 'fromCity', 'isAvailable', 'views', 'updates']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("4_top.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [29]:
def get_stats_for_categories(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        category, SUM({col}), MIN({col}), MAX({col}), AVG({col}), COUNT(*)  
        FROM DataTable4 GROUP BY category""")
    return stats.fetchall()

stats = get_stats_for_categories(conn, 'price')
keys = ['category', 'sum', 'min', 'max', 'avg', 'count']

with open("4_stats_for_categories.json", "w", encoding="utf-8") as outfile: 
    json.dump([{keys[i]: str(e) for i, e in enumerate(category)} for category in stats], outfile, ensure_ascii=True)

In [30]:
def get_stats(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        SUM({col}), MIN({col}), MAX({col}), AVG({col})  
        FROM DataTable4""")
    return stats.fetchone()

stats = get_stats(conn, 'price')
keys = ['sum', 'min', 'max', 'avg']

with open("4_stats_.json", "w", encoding="utf-8") as outfile: 
    json.dump({keys[i]: str(e) for i, e in enumerate(stats)}, outfile, ensure_ascii=True)

In [31]:
def get_freq(conn, col):
    cur = conn.cursor()
    freq = cur.execute(f"""SELECT 
        {col}, COUNT(*)
        FROM DataTable4 GROUP BY {col}""")
    return freq

freq = get_freq(conn, 'updates')
freq = {e[0]: str(e[1]) for e in freq}

with open("4_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

# Задание 5

In [32]:
import sqlite3
import pandas as pd

# anime = pd.read_csv('./data/anime.csv').replace('Unknown', np.nan).dropna()
# anime.to_csv('./data/anime.csv', index=False)
anime = pd.read_csv('./data/anime.csv')

# ws = pd.read_csv('./data/animelist.csv')
# user_ids = ws['user_id'].unique()[:1000]
# ws = ws[ws.user_id.isin(user_ids)]
# ws.to_csv('./data/animelist.csv', index=False)
ws = pd.read_csv('./data/animelist.csv')

In [33]:
def create_table(conn, sql):
    cur = conn.cursor()
    cur.execute(sql)

In [34]:
conn = sqlite3.connect('database_custom.db')
create_table(conn, """CREATE TABLE AnimeInfo (
        id INTEGER,
        title TEXT,
        score TEXT,
        genres TEXT,
        type TEXT,
        episodes INTEGER, 
        producers TEXT,
        rating TEXT
        )""")

create_table(conn, """CREATE TABLE AnimeStats (
        id INTEGER,
        members INTEGER,
        favorites INTEGER,
        watching INTEGER,
        completed INTEGER,
        on_hold INTEGER,
        dropped INTEGER,
        plan_to_watch INTEGER,
        score_10 FLOAT,
        score_9 FLOAT,
        score_8 FLOAT,
        score_7 FLOAT,
        score_6 FLOAT,
        score_5 FLOAT,
        score_4 FLOAT,
        score_3 FLOAT,
        score_2 FLOAT,
        score_1 FLOAT
        )""")

create_table(conn, """CREATE TABLE UserRating (
        user_id INTEGER,
        anime_id INTEGER,
        rating INTEGER,
        watching_status INTEGER
        )""")

In [35]:
def parse_df(df, columns, res_keys):
    data = df[columns].to_dict()
    keys = list(data.keys())
    n = len(data[keys[0]])
    return [{res_keys[i] : data[key][j] for i, key in enumerate(keys)} for j in range(n)]


def insert_data(conn, data, sql):
    cur = conn.cursor()
    cur.executemany(sql, data)
    conn.commit()

In [36]:
anime_info_columns = ['MAL_ID', 'Name', 'Score', 'Genres', 'Type','Episodes', 'Producers','Rating']
anime_info_keys = ['id', 'title', 'score', 'genres', 'type', 'episodes', 'producers', 'rating']
anime_stats_columns = ['MAL_ID', 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped', 
                       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5',
                       'Score-4', 'Score-3', 'Score-2', 'Score-1']
anime_stats_keys = ['id', 'members', 'favorites', 'watching', 'completed', 'on_hold', 
                    'dropped', 'plan_to_watch', 'score_10', 'score_9', 'score_8', 'score_7', 
                    'score_6', 'score_5', 'score_4', 'score_3', 'score_2', 'score_1']
user_rating_columns = ['user_id', 'anime_id', 'rating', 'watching_status']
user_rating_keys = ['user_id', 'anime_id', 'rating', 'watching_status']

data1 = parse_df(anime, anime_info_columns, anime_info_keys)
data2 = parse_df(anime, anime_stats_columns, anime_stats_keys)
data3 = parse_df(ws, user_rating_columns, user_rating_keys)

insert_data(conn, 
            data1, 
            """
                INSERT INTO AnimeInfo (id, title, score, genres, type, episodes, producers, rating)
                VALUES(
                    :id, :title, :score, :genres, :type, :episodes, :producers, :rating
                )
            """)

insert_data(conn, 
            data2, 
            """
                INSERT INTO AnimeStats (id, members, favorites, watching, completed, on_hold, 
                dropped, plan_to_watch, score_10, score_9, score_8, score_7, score_6, score_5, 
                score_4, score_3, score_2, score_1)
                VALUES(
                    :id, :members, :favorites, :watching, :completed, :on_hold, 
                    :dropped, :plan_to_watch, :score_10, :score_9, :score_8, :score_7, :score_6, :score_5, 
                    :score_4, :score_3, :score_2, :score_1
                )
            """)

insert_data(conn, 
            data3, 
            """
                INSERT INTO UserRating (user_id, anime_id, rating, watching_status)
                VALUES(
                    :user_id, :anime_id, :rating, :watching_status
                )
            """)

In [37]:
# 1
def get_from_head_with_condition(conn, sort_col, n, condition):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT title,
        score,
        producers
        FROM AnimeInfo WHERE {condition} ORDER BY {sort_col} DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head_with_condition(conn, 'rating', 12, 'score > 8')

keys = ['title', 'author', 'genre', 'pages', 'published_year', 'isbn', 'rating', 'views']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("5_top_with_condition.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [38]:
# 2
def get_freq(conn, col):
    cur = conn.cursor()
    freq = cur.execute(f"""SELECT 
        {col}, COUNT(*)
        FROM UserRating GROUP BY {col}""")
    return freq

freq = get_freq(conn, 'watching_status')
freq = {e[0]: str(e[1]) for e in freq}

with open("5_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump(freq, outfile, ensure_ascii=True)

In [39]:
# 3
def get_from_head_both_ordered(conn, n, sort_col):
    cur = conn.cursor()
    samples = cur.execute(f"""SELECT AnimeInfo.title,
        AnimeInfo.producers,
        UserRating.watching_status
        FROM AnimeInfo JOIN UserRating ON AnimeInfo.id = UserRating.anime_id
        ORDER BY {sort_col}
        DESC LIMIT {str(n)}""")
    return samples


cur = get_from_head_both_ordered(conn, 12, 'UserRating.rating')

keys = ['title', 'producers', 'watching_status']
result = []
for row in cur.fetchall():
    result.append({keys[i]: str(e) for i, e in enumerate(row)})
    
with open("5_top_from_both_ordered.json", "w", encoding="utf-8") as outfile: 
    json.dump(result, outfile, ensure_ascii=True)

In [40]:
# 4
def delete_by_col(conn, column, value):
    cur = conn.cursor()
    cur.execute(f"DELETE FROM UserRating WHERE {column} = {value}")
    conn.commit()

delete_by_col(conn, 'watching_status', '0')

In [41]:
# 5
def get_stats(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        SUM({col}), MIN({col}), MAX({col}), AVG({col})  
        FROM AnimeInfo""")
    return stats.fetchone()

stats = get_stats(conn, 'score')
keys = ['sum', 'min', 'max', 'avg']

with open("5_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump({keys[i]: str(e) for i, e in enumerate(stats)}, outfile, ensure_ascii=True)

In [42]:
# 6
def get_stats_for_categories(conn, col):
    cur = conn.cursor()
    stats = cur.execute(f"""SELECT 
        rating, SUM({col}), MIN({col}), MAX({col}), AVG({col}), COUNT(*)  
        FROM AnimeInfo GROUP BY rating""")
    return stats.fetchall()

stats = get_stats_for_categories(conn, 'score')
keys = ['category', 'sum', 'min', 'max', 'avg', 'count']

with open("5_stats_for_categories.json", "w", encoding="utf-8") as outfile: 
    json.dump([{keys[i]: str(e) for i, e in enumerate(category)} for category in stats], outfile, ensure_ascii=True)